Gonna fix up the features in this noteboook: 

I need to add more features to the dataset to make the models be able to predict better. Right now the variation is not explained but the current number of features. There are other covariates that play a role, and for some of them I have data for.  

In this notebook, I will figure out how to incorporate these features into one big dataset that I can use with models. 


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

hero_data = pd.read_csv("trimmed_hero_data.csv")
map_winrate = pd.read_csv("hero_map_winrate.csv")
mode_winrate = pd.read_csv("hero_mode_winrate.csv")

hero_data.head()

,player_id,hero_name,role,timePlayed_value,matchesWinPct_value,matchesPlayed_value,kdRatio_value,kdaRatio_value,totalHeroDamagePerMinute_value,totalHeroHealPerMinute_value,totalDamageTakenPerMinute_value,lastKills_value,totalMvp_value,totalSvp_value,percent_of_time_played
0,VP Sypeh,Gambit,strategist,37388.742,69.135802,48.6,4.075556,9.626667,881.0,2983.0,682.0,280,18,7,55.159494
1,VP Sypeh,Invisible Woman,strategist,14769.481,63.934426,18.3,4.962500,11.475000,1294.0,3169.0,1516.0,79,7,5,21.789369
2,VP Sypeh,Emma Frost,vanguard,1275.047,71.428571,1.4,3.363636,4.545455,2404.0,0.0,3061.0,13,0,0,1.881073
3,VP Sypeh,Loki,strategist,7069.450,93.220339,11.8,5.400000,13.400000,957.0,2267.0,511.0,31,4,0,10.429537
4,VP Sypeh,Adam Warlock,strategist,2216.817,53.333333,3.0,2.750000,4.850000,1272.0,1596.0,727.0,28,0,0,3.270463


In [2]:
map_winrate.head()

,player_id,hero_name,map_name,map_winrate
0,-oo- <---MyPig,Angela,Arakko,1.0
1,-oo- <---MyPig,Angela,Central Park,1.0
2,-oo- <---MyPig,Angela,Heart Of Heaven,1.0
3,-oo- <---MyPig,Angela,Hell's Heaven,1.0
4,-oo- <---MyPig,Angela,Spider-Islands,0.5


In [3]:
mode_winrate.head()

,player_id,hero_name,map_mode,mode_winrate
0,-oo- <---MyPig,Angela,Convergence,1.000000
1,-oo- <---MyPig,Angela,Convoy,0.666667
2,-oo- <---MyPig,Angela,Domination,1.000000
3,-oo- <---MyPig,Emma Frost,Convergence,0.666667
4,-oo- <---MyPig,Emma Frost,Convoy,0.333333


I can take the winrates per map and per mode, and make each specific map and each specific mode its own column. I can then use this to enter the corresponding value. 

For missing winrates, in the case a player has not played a hero on a map, I think it is safe to put it at either 50% or 0%. I think 50% is fair because it is equally likely to win or lose if no information is known, however that could be bad because that would mean that somehow playing a hero on a map for the first time is higher chances of winning than a hero on a map that theyve played a lot and know the layout of? That is confusing. Maybe it is best to assume that they will lose, but that would be 0%, so maybe 50% is most fair afterall. BUT, the other issue is if a player is very good with a player, and has high stats on every map, then why would i default to 50%? Because it isnt likely the player has never played on that map at all, just with that hero. So the more safer pick would probably be to just use the player's overall win rate % for a hero, because that is the average performance on that hero through consistent data, so this is safer than saying 50% (say, they are REALLY bad with a hero, and overall win % is 5%, or really good and win rate is 70%, then it wouldnt be fair to say 50%). 

need to make pivot tables to make the map names and map modes values into column names 
the player id and hero name cols should stay as is 

In [4]:
map_winrate_PIVOT = map_winrate.pivot_table(index=['player_id', 'hero_name'], columns='map_name', values='map_winrate').reset_index()
mode_winrate_PIVOT = mode_winrate.pivot_table(index=['player_id', 'hero_name'], columns='map_mode', values='mode_winrate').reset_index()
map_winrate_PIVOT

map_name,player_id,hero_name,Arakko,Birnin T'Challa,Celestial Husk,Central Park,Hall Of Djalia,Heart Of Heaven,Hell's Heaven,Krakoa,Midtown,Royal Palace,Spider-Islands,Symbiotic Surface,Yggdrasill Path
0,-oo- <---MyPig,Angela,1.0,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,0.5,NaN,NaN
1,-oo- <---MyPig,Emma Frost,NaN,0.5,NaN,0.0,NaN,1.0,0.0,NaN,NaN,NaN,1.0,1.0,0.000000
2,-oo- <---MyPig,Groot,NaN,0.0,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,1.0,0.000000
3,-oo- <---MyPig,Magneto,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-oo- <---MyPig,Wolverine,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1835,斩尽杀绝,Psylocke,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1836,이페,Adam Warlock,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1837,이페,Gambit,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,0.0,1.0,0.666667
1838,이페,Invisible Woman,NaN,0.0,1.0,1.0,NaN,0.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN


In [5]:
mode_winrate_PIVOT

map_mode,player_id,hero_name,Convergence,Convoy,Domination
0,-oo- <---MyPig,Angela,1.000000,0.666667,1.000000
1,-oo- <---MyPig,Emma Frost,0.666667,0.333333,0.333333
2,-oo- <---MyPig,Groot,1.000000,0.000000,0.000000
3,-oo- <---MyPig,Magneto,NaN,NaN,1.000000
4,-oo- <---MyPig,Wolverine,NaN,NaN,0.000000
...,...,...,...,...,...
1835,斩尽杀绝,Psylocke,1.000000,NaN,NaN
1836,이페,Adam Warlock,NaN,NaN,1.000000
1837,이페,Gambit,1.000000,0.500000,1.000000
1838,이페,Invisible Woman,0.500000,1.000000,0.833333


Based on the logic I thought through above, I will fill in NaN values iwth the average hero win rates for each player. But that will come after I first merge these tables together with the main hero table first

In [6]:
#easy merge using both map and mode winrates on left join 
hero_data_w_map_mode = hero_data.merge(map_winrate_PIVOT, on=['player_id', 'hero_name'], how='left').merge(mode_winrate_PIVOT, on=['player_id', 'hero_name'], how='left')
hero_data_w_map_mode

,player_id,hero_name,role,timePlayed_value,matchesWinPct_value,matchesPlayed_value,kdRatio_value,kdaRatio_value,totalHeroDamagePerMinute_value,totalHeroHealPerMinute_value,...,Hell's Heaven,Krakoa,Midtown,Royal Palace,Spider-Islands,Symbiotic Surface,Yggdrasill Path,Convergence,Convoy,Domination
0,VP Sypeh,Gambit,strategist,37388.742,69.135802,48.6,4.075556,9.626667,881.0,2983.0,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.000000,1.0,0.000000
1,VP Sypeh,Invisible Woman,strategist,14769.481,63.934426,18.3,4.962500,11.475000,1294.0,3169.0,...,0.5,0.666667,1.0,NaN,1.0,1.0,NaN,0.666667,1.0,0.714286
2,VP Sypeh,Emma Frost,vanguard,1275.047,71.428571,1.4,3.363636,4.545455,2404.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VP Sypeh,Loki,strategist,7069.450,93.220339,11.8,5.400000,13.400000,957.0,2267.0,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.000000,1.0,1.000000
4,VP Sypeh,Adam Warlock,strategist,2216.817,53.333333,3.0,2.750000,4.850000,1272.0,1596.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3952,DecaDonald,Loki,strategist,37808.155,57.201646,48.6,2.886207,7.500000,701.0,2287.0,...,0.0,NaN,1.0,NaN,NaN,NaN,1.0,1.000000,1.0,0.000000
3953,DecaDonald,Invisible Woman,strategist,9567.059,33.628319,11.3,2.098901,5.571429,677.0,2592.0,...,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
3954,DecaDonald,Rocket Raccoon,strategist,13100.055,59.067358,19.3,1.400000,6.410526,207.0,1988.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,1.000000
3955,DecaDonald,Venom,vanguard,1613.943,40.000000,2.5,2.117647,2.941176,1095.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000


In [7]:
#fill nans with 'matchesWinPct_value' for all map and mode columns 
#get all cols in list
cols = mode_winrate_PIVOT.columns.tolist()
cols.remove('player_id')
cols.remove('hero_name')
cols.extend(map_winrate_PIVOT.columns.tolist()) #have to use extend to add list items to list, instead of appending the physical list as an item to the list 
cols.remove('player_id')
cols.remove('hero_name')
print(cols)

#im actually going to change the win% column to a decimal because this is getting ridiculous
hero_data_w_map_mode['matchesWinPct_value'] = hero_data_w_map_mode['matchesWinPct_value'] 




['Convergence', 'Convoy', 'Domination', 'Arakko', "Birnin T'Challa", 'Celestial Husk', 'Central Park', 'Hall Of Djalia', 'Heart Of Heaven', "Hell's Heaven", 'Krakoa', 'Midtown', 'Royal Palace', 'Spider-Islands', 'Symbiotic Surface', 'Yggdrasill Path']


*****
MAJOR CHANGE HERE: 

Originally, I was filling NaN values with the match win %, but this caused models to learn incorrectly and end up learning nothing and get 100% accuracy, since the match win % is what is supposed to be predicted. 

This time, finding the overall average for the match win % for the whole data set and putting that in instead. This is an unbiased value that is fair, and the model wont learn incorrectly. 
****

In [8]:
#for col in cols: 
  #  hero_data_w_map_mode[col] = hero_data_w_map_mode[col].fillna((hero_data_w_map_mode['matchesWinPct_value'])/100)

average_win_pct_overall = hero_data_w_map_mode['matchesWinPct_value'].mean()
for col in cols: 
    hero_data_w_map_mode[col] = hero_data_w_map_mode[col].fillna(average_win_pct_overall)

also, I am a bit annoyed at the time played being in seconds, the numbers are too big, I'm dividing by 3600 to get it in hours, as thats more telling, i cant even interpret seconds appropriately

In [9]:
hero_data_w_map_mode['timePlayed_value'] = hero_data_w_map_mode['timePlayed_value'] / 3600  
hero_data_w_map_mode

,player_id,hero_name,role,timePlayed_value,matchesWinPct_value,matchesPlayed_value,kdRatio_value,kdaRatio_value,totalHeroDamagePerMinute_value,totalHeroHealPerMinute_value,...,Hell's Heaven,Krakoa,Midtown,Royal Palace,Spider-Islands,Symbiotic Surface,Yggdrasill Path,Convergence,Convoy,Domination
0,VP Sypeh,Gambit,strategist,10.385762,69.135802,48.6,4.075556,9.626667,881.0,2983.0,...,56.461653,56.461653,1.000000,56.461653,56.461653,56.461653,56.461653,1.000000,1.000000,0.000000
1,VP Sypeh,Invisible Woman,strategist,4.102634,63.934426,18.3,4.962500,11.475000,1294.0,3169.0,...,0.500000,0.666667,1.000000,56.461653,1.000000,1.000000,56.461653,0.666667,1.000000,0.714286
2,VP Sypeh,Emma Frost,vanguard,0.354180,71.428571,1.4,3.363636,4.545455,2404.0,0.0,...,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653
3,VP Sypeh,Loki,strategist,1.963736,93.220339,11.8,5.400000,13.400000,957.0,2267.0,...,56.461653,56.461653,56.461653,56.461653,56.461653,1.000000,1.000000,1.000000,1.000000,1.000000
4,VP Sypeh,Adam Warlock,strategist,0.615783,53.333333,3.0,2.750000,4.850000,1272.0,1596.0,...,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3952,DecaDonald,Loki,strategist,10.502265,57.201646,48.6,2.886207,7.500000,701.0,2287.0,...,0.000000,56.461653,1.000000,56.461653,56.461653,56.461653,1.000000,1.000000,1.000000,0.000000
3953,DecaDonald,Invisible Woman,strategist,2.657516,33.628319,11.3,2.098901,5.571429,677.0,2592.0,...,56.461653,0.000000,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,0.000000
3954,DecaDonald,Rocket Raccoon,strategist,3.638904,59.067358,19.3,1.400000,6.410526,207.0,1988.0,...,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,0.000000,56.461653,1.000000
3955,DecaDonald,Venom,vanguard,0.448317,40.000000,2.5,2.117647,2.941176,1095.0,0.0,...,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,1.000000


Im also gonna change win % to a decimal

In [10]:
hero_data_w_map_mode['matchesWinPct_value'] = hero_data_w_map_mode['matchesWinPct_value'] / 100
hero_data_w_map_mode

,player_id,hero_name,role,timePlayed_value,matchesWinPct_value,matchesPlayed_value,kdRatio_value,kdaRatio_value,totalHeroDamagePerMinute_value,totalHeroHealPerMinute_value,...,Hell's Heaven,Krakoa,Midtown,Royal Palace,Spider-Islands,Symbiotic Surface,Yggdrasill Path,Convergence,Convoy,Domination
0,VP Sypeh,Gambit,strategist,10.385762,0.691358,48.6,4.075556,9.626667,881.0,2983.0,...,56.461653,56.461653,1.000000,56.461653,56.461653,56.461653,56.461653,1.000000,1.000000,0.000000
1,VP Sypeh,Invisible Woman,strategist,4.102634,0.639344,18.3,4.962500,11.475000,1294.0,3169.0,...,0.500000,0.666667,1.000000,56.461653,1.000000,1.000000,56.461653,0.666667,1.000000,0.714286
2,VP Sypeh,Emma Frost,vanguard,0.354180,0.714286,1.4,3.363636,4.545455,2404.0,0.0,...,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653
3,VP Sypeh,Loki,strategist,1.963736,0.932203,11.8,5.400000,13.400000,957.0,2267.0,...,56.461653,56.461653,56.461653,56.461653,56.461653,1.000000,1.000000,1.000000,1.000000,1.000000
4,VP Sypeh,Adam Warlock,strategist,0.615783,0.533333,3.0,2.750000,4.850000,1272.0,1596.0,...,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3952,DecaDonald,Loki,strategist,10.502265,0.572016,48.6,2.886207,7.500000,701.0,2287.0,...,0.000000,56.461653,1.000000,56.461653,56.461653,56.461653,1.000000,1.000000,1.000000,0.000000
3953,DecaDonald,Invisible Woman,strategist,2.657516,0.336283,11.3,2.098901,5.571429,677.0,2592.0,...,56.461653,0.000000,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,0.000000
3954,DecaDonald,Rocket Raccoon,strategist,3.638904,0.590674,19.3,1.400000,6.410526,207.0,1988.0,...,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,0.000000,56.461653,1.000000
3955,DecaDonald,Venom,vanguard,0.448317,0.400000,2.5,2.117647,2.941176,1095.0,0.0,...,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,56.461653,1.000000


In [11]:
hero_data_w_map_mode.to_csv("hero_data_with_map_mode_winrates.csv", index=False)

*****
So now I want to try and incorporate the ideas of synergies and opponent counter picks. 
*****

So counter picks is a hard to incorporate thing, as yes there are counters, but the level/degree someone is countered by a character is not really quantified.  

So I might actually have to leave counter picking out for now, as this is also a bit of information you can only have in the middle of a match that's already in progress. Also all heroes have some other hero that they are weak to, or multiple, so it shouldn't influence the decision to continue to play a hero in the long run because it is an affect that widely distributed fairly evenly. 

However, team synergies is definitely something that could be taken into consideration. This might be awkward to incorporate though. 

Team synergies could just be something that I count after creating a list of all the synergies currently available in the game (specifically through team up abilities that provide an advantage), but this is a variable that is affected by other people as well. 

The other thing is just in general synergy between heroes that might not even have a teamup. HOWEVER, i have decided not to try and do this right now as there are 44 heroes in the game, and in general to analyze that, i would have to consider each individual pairing between one hero and another, which would be 44x44 = 1,936 hero pairings in total, and to see win rates with individual pairings I would probably have to end up adding 1,936 additional columns, and I do not know how NaN values would be handled (maybe a 0?). However, given the total data being kind of low, and the high number of possibilities, most of the data will likely be 0, 0.5, or 1. 

So instead of that, I do know from general trends that the specifically provided hero synergies in the game through teamups, which gives an advantage through additional abilities, are more often chosen than others. There are also less than 20 built in synergies, so that would NOT be detrimental to add in an additional less than 20 columns, and they are much more common so that is actually useful. 


So, perhaps a feature I can do is for a certain hero, I can see what the win rate was with synergies and what the win rate was without? 


Another thing I realized: 
For actually figuring out all possible hero synergies, I do not need 1,936 columns, I only need 43 additional columns with hero synergy data for that player+hero combo. So it only adds 43 columns which is completely fine. 

I already have actual full match/scoreboard data ready to go in the format where I have the player+hero w/ team comp, opponent comp, and win/loss stat. I should be able to do this below: 



In [12]:
scoreboard_data = pd.read_csv("matches_cleaned.csv")
team_comp_winrates = pd.read_csv("team_comp_win_rates.csv")
scoreboard_data

,match_id,player_id,team_id,won,hero_name,hero_id,map_name,match_mode,duration_sec,is_mvp,...,kdaRatio_value,totalHeroDamage_value,totalHeroDamagePerMinute_value,totalHeroHeal_value,totalHeroHealPerMinute_value,totalDamageTaken_value,totalDamageTakenPerMinute_value,lastKills_value,headKills_value,soloKills_value
0,5513730_1765074019_1318548_11001_12,Deleted404,0,1,Rocket Raccoon,1023,Celestial Husk,Domination,381,False,...,10.000000,655.769723,109.29,11943.727504,1990.62,2981.606651,496.93,2,0,0
1,5513730_1765074019_1318548_11001_12,Kaifs,0,1,Magneto,1037,Celestial Husk,Domination,381,False,...,13.000000,7781.313597,1296.89,0.000000,0.00,11832.338579,1972.06,4,0,1
2,5513730_1765074019_1318548_11001_12,Finnsi,0,1,Groot,1027,Celestial Husk,Domination,381,False,...,6.500000,8685.591116,1447.60,0.000000,0.00,19402.304778,3233.72,6,0,1
3,5513730_1765074019_1318548_11001_12,JukeBox2274,1,0,Angela,1056,Celestial Husk,Domination,381,False,...,0.666667,6113.298075,1018.88,0.000000,0.00,18688.655201,3114.78,1,0,1
4,5513730_1765074019_1318548_11001_12,VP Sypeh,0,1,Invisible Woman,1050,Celestial Husk,Domination,381,False,...,27.000000,4548.798798,758.13,14847.611123,2474.60,6147.800565,1024.63,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64051,5513925_1765094719_1231224_11001_12,iffvy,1,0,Winter Soldier,1041,Yggdrasill Path,Convoy,1059,False,...,3.818182,32969.463813,1939.38,2477.660993,145.74,19305.615006,1135.62,24,0,1
64052,5513925_1765094719_1231224_11001_12,Rajahyrax,1,0,Groot,1027,Yggdrasill Path,Convoy,1059,False,...,4.555556,28770.961880,1692.41,0.000000,0.00,82469.576583,4851.15,7,0,1
64053,5513925_1765094719_1231224_11001_12,Пyк,1,0,Rocket Raccoon,1023,Yggdrasill Path,Convoy,1059,False,...,7.285714,5906.691635,347.45,39778.870284,2339.93,12175.875208,716.23,1,0,0
64054,5513925_1765094719_1231224_11001_12,FAZE BLAZIKEN,0,1,Phoenix,1054,Yggdrasill Path,Convoy,1059,False,...,3.200000,38203.973442,2247.29,1665.143768,97.95,15347.804127,902.81,13,1,2


team_comp_winrates

What if i grouped by player id + hero name like in the other tables, then calculated the synergies

scoreboard_data_grouped = scoreboard_data.groupby(['player_id', 'hero_name'])
scoreboard_data_grouped

Making another version of the team composition + win rate table but making sure to include the player id too

In [13]:
team_row_separating = scoreboard_data[['match_id', 'player_id' ,'team_id', 'hero_name', 'won']]
team_row_separating

,match_id,player_id,team_id,hero_name,won
0,5513730_1765074019_1318548_11001_12,Deleted404,0,Rocket Raccoon,1
1,5513730_1765074019_1318548_11001_12,Kaifs,0,Magneto,1
2,5513730_1765074019_1318548_11001_12,Finnsi,0,Groot,1
3,5513730_1765074019_1318548_11001_12,JukeBox2274,1,Angela,0
4,5513730_1765074019_1318548_11001_12,VP Sypeh,0,Invisible Woman,1
...,...,...,...,...,...
64051,5513925_1765094719_1231224_11001_12,iffvy,1,Winter Soldier,0
64052,5513925_1765094719_1231224_11001_12,Rajahyrax,1,Groot,0
64053,5513925_1765094719_1231224_11001_12,Пyк,1,Rocket Raccoon,0
64054,5513925_1765094719_1231224_11001_12,FAZE BLAZIKEN,0,Phoenix,1


In [14]:
#have to sort during the lambda function otherwise the same heroes in different orders will be treated as different comps
#grouping by the match and the 2 teams, and then for each team aggregating the heroes and ordering them w/ comma separation
team_comps = team_row_separating.groupby(['match_id', 'team_id']).agg({'hero_name': lambda x: ', '.join(sorted(x)),
'won': 'max'}).reset_index() 
team_comps = team_comps.reset_index()
team_comps

,index,match_id,team_id,hero_name,won
0,0,4859912_1764413878_2042276_11001_32,0,"Gambit, Invisible Woman, Magneto, Phoenix, Spi...",1
1,1,4859912_1764413878_2042276_11001_32,1,"Gambit, Invisible Woman, Magneto, Spider-Man, ...",0
2,2,4859918_1764785795_1291050_11001_32,0,"Daredevil, Gambit, Luna Snow, Magneto, Venom, ...",0
3,3,4859918_1764785795_1291050_11001_32,1,"Gambit, Iron Fist, Magneto, Phoenix, Rocket Ra...",1
4,4,4859927_1764258769_1291106_11001_32,0,"Blade, Emma Frost, Invisible Woman, Magneto, R...",1
...,...,...,...,...,...
10671,10671,6713333_1763758119_1267033_11001_22,1,"Black Widow, Gambit, Invisible Woman, Magneto,...",1
10672,10672,6713340_1764966783_2042019_11001_22,0,"Black Panther, Emma Frost, Invisible Woman, Lu...",0
10673,10673,6713340_1764966783_2042019_11001_22,1,"Angela, Doctor Strange, Hawkeye, Invisible Wom...",1
10674,10674,6713853_1736615457_110_11001_10,0,"Invisible Woman, Iron Man, Mantis, Mister Fant...",1


going to create a dictionary style groupby object where i can look up the full team based on match id + team id, so match x and team 0 will be the list of all heros in team 0, and match x and team 1 is the list of all heroes in team 1

In [15]:
team_comps_dictionary = team_row_separating.groupby(['match_id', 'team_id'])['hero_name'].apply(lambda x: ', '.join(sorted(x.tolist()))).to_dict()
team_comps_dictionary


{('4859912_1764413878_2042276_11001_32',
  0): 'Gambit, Invisible Woman, Magneto, Phoenix, Spider-Man, Thor',
 ('4859912_1764413878_2042276_11001_32',
  1): 'Gambit, Invisible Woman, Magneto, Spider-Man, The Punisher, Venom',
 ('4859918_1764785795_1291050_11001_32',
  0): 'Daredevil, Gambit, Luna Snow, Magneto, Venom, Winter Soldier',
 ('4859918_1764785795_1291050_11001_32',
  1): 'Gambit, Iron Fist, Magneto, Phoenix, Rocket Raccoon, Venom',
 ('4859927_1764258769_1291106_11001_32',
  0): 'Blade, Emma Frost, Invisible Woman, Magneto, Rocket Raccoon, The Punisher',
 ('4859927_1764258769_1291106_11001_32',
  1): 'Hela, Hela, Invisible Woman, Luna Snow, Phoenix, Winter Soldier',
 ('4859930_1764788081_1292175_11001_32',
  0): 'Angela, Cloak & Dagger, Iron Fist, Luna Snow, Magneto, Winter Soldier',
 ('4859930_1764788081_1292175_11001_32',
  1): 'Cloak & Dagger, Emma Frost, Luna Snow, Magneto, Phoenix, Winter Soldier',
 ('4859940_1764432146_1310523_11001_32',
  0): 'Daredevil, Invisible Woman

next i make a new column listing all team heroes for a team, based on the match id and team id thats already listed in the row

In [16]:
team_row_separating['team_heroes'] = team_row_separating.apply(lambda x: team_comps_dictionary[(x['match_id'], x['team_id'])], axis=1)
team_row_separating

C:\Users\Nazir\AppData\Local\Temp\ipykernel_35760\1652149225.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_row_separating['team_heroes'] = team_row_separating.apply(lambda x: team_comps_dictionary[(x['match_id'], x['team_id'])], axis=1)


,match_id,player_id,team_id,hero_name,won,team_heroes
0,5513730_1765074019_1318548_11001_12,Deleted404,0,Rocket Raccoon,1,"Groot, Hela, Invisible Woman, Magik, Magneto, ..."
1,5513730_1765074019_1318548_11001_12,Kaifs,0,Magneto,1,"Groot, Hela, Invisible Woman, Magik, Magneto, ..."
2,5513730_1765074019_1318548_11001_12,Finnsi,0,Groot,1,"Groot, Hela, Invisible Woman, Magik, Magneto, ..."
3,5513730_1765074019_1318548_11001_12,JukeBox2274,1,Angela,0,"Angela, Emma Frost, Hela, Jeff The Land Shark,..."
4,5513730_1765074019_1318548_11001_12,VP Sypeh,0,Invisible Woman,1,"Groot, Hela, Invisible Woman, Magik, Magneto, ..."
...,...,...,...,...,...,...
64051,5513925_1765094719_1231224_11001_12,iffvy,1,Winter Soldier,0,"Groot, Invisible Woman, Phoenix, Rocket Raccoo..."
64052,5513925_1765094719_1231224_11001_12,Rajahyrax,1,Groot,0,"Groot, Invisible Woman, Phoenix, Rocket Raccoo..."
64053,5513925_1765094719_1231224_11001_12,Пyк,1,Rocket Raccoon,0,"Groot, Invisible Woman, Phoenix, Rocket Raccoo..."
64054,5513925_1765094719_1231224_11001_12,FAZE BLAZIKEN,0,Phoenix,1,"Bruce Banner, Invisible Woman, Magneto, Phoeni..."


Now that I have the full team comp provided for each player_id and the hero_name of the hero they played, I could try to map out associations with hero played for a player and which teammates they played with to get a win

I want to get every player id and hero in a row, along with every single teammate hero. That way, i can then just get the averages right away for the won games. 

In [17]:
rows_for_every_teammate = []
#for each row in the dataframe, collecting the player name, hero they played, win/loss, and then breaking apart 
#the teammates list and creating a new row for each of the teammates with this information repeated 
for index, row in team_row_separating.iterrows(): #break apart row
    teammates = row['team_heroes'].split(', ') #selecct specifically the column with the hero list of teammates
    player_name = row['player_id']
    hero_name = row['hero_name']
    win_loss = row['won']

    for hero in teammates:
        if hero != row['hero_name']:  #exclude the player's own hero from their teammates
            new_row = { #craft that new row 
                'player_id': player_name,
                'hero_name': hero_name,
                'teammate_hero': hero,
                'won': win_loss
            }
            rows_for_every_teammate.append(new_row) #add it to full row master list 


teammate_hero_df = pd.DataFrame(rows_for_every_teammate) 
teammate_hero_df

,player_id,hero_name,teammate_hero,won
0,Deleted404,Rocket Raccoon,Groot,1
1,Deleted404,Rocket Raccoon,Hela,1
2,Deleted404,Rocket Raccoon,Invisible Woman,1
3,Deleted404,Rocket Raccoon,Magik,1
4,Deleted404,Rocket Raccoon,Magneto,1
...,...,...,...,...
319655,сountry mouse,Phoenix,Groot,0
319656,сountry mouse,Phoenix,Invisible Woman,0
319657,сountry mouse,Phoenix,Rocket Raccoon,0
319658,сountry mouse,Phoenix,Thor,0


groupby all win/loss stats per hero

In [18]:
#group the other 3 cols with the win column 
win_loss_teammate_hero_stats = teammate_hero_df.groupby(['player_id', 'hero_name', 'teammate_hero'])['won']
print(win_loss_teammate_hero_stats)


this should have specifically the target data grouped by win/loss stats. Can now find the average win rate for each combination and the total games played together as well

In [19]:
win_loss_teammate_hero_MEAN = win_loss_teammate_hero_stats.mean().reset_index()
win_loss_teammate_hero_MEAN

,player_id,hero_name,teammate_hero,won
0,' F ',Doctor Strange,Angela,1.000000
1,' F ',Doctor Strange,Loki,1.000000
2,' F ',Doctor Strange,Mantis,1.000000
3,' F ',Doctor Strange,Spider-Man,1.000000
4,' F ',Doctor Strange,Wolverine,1.000000
...,...,...,...,...
198843,ｚаｚа,Venom,Mantis,1.000000
198844,ｚаｚа,Venom,Phoenix,0.333333
198845,ｚаｚа,Venom,Psylocke,0.000000
198846,ｚаｚа,Venom,Rocket Raccoon,0.000000


In [20]:
win_loss_teammate_hero_MEAN.rename(columns={'won': 'teammate_hero_winrate'}, inplace=True)

In [21]:
win_loss_teammate_hero_COUNT = win_loss_teammate_hero_stats.count().reset_index()
win_loss_teammate_hero_COUNT.rename(columns={'won': 'teammate_hero_games_played'}, inplace=True)
win_loss_teammate_hero_COUNT


,player_id,hero_name,teammate_hero,teammate_hero_games_played
0,' F ',Doctor Strange,Angela,1
1,' F ',Doctor Strange,Loki,1
2,' F ',Doctor Strange,Mantis,1
3,' F ',Doctor Strange,Spider-Man,1
4,' F ',Doctor Strange,Wolverine,1
...,...,...,...,...
198843,ｚаｚа,Venom,Mantis,1
198844,ｚаｚа,Venom,Phoenix,3
198845,ｚаｚа,Venom,Psylocke,3
198846,ｚаｚа,Venom,Rocket Raccoon,2


merging it all together 

In [22]:
teammate_hero_df_w_synergy = teammate_hero_df.merge(win_loss_teammate_hero_MEAN, on=['player_id', 'hero_name', 'teammate_hero'], how='left').merge(win_loss_teammate_hero_COUNT, on=['player_id', 'hero_name', 'teammate_hero'], how='left')
teammate_hero_df_w_synergy

,player_id,hero_name,teammate_hero,won,teammate_hero_winrate,teammate_hero_games_played
0,Deleted404,Rocket Raccoon,Groot,1,0.333333,3
1,Deleted404,Rocket Raccoon,Hela,1,0.833333,6
2,Deleted404,Rocket Raccoon,Invisible Woman,1,0.500000,4
3,Deleted404,Rocket Raccoon,Magik,1,1.000000,2
4,Deleted404,Rocket Raccoon,Magneto,1,0.727273,11
...,...,...,...,...,...,...
319655,сountry mouse,Phoenix,Groot,0,0.000000,1
319656,сountry mouse,Phoenix,Invisible Woman,0,0.000000,1
319657,сountry mouse,Phoenix,Rocket Raccoon,0,0.000000,1
319658,сountry mouse,Phoenix,Thor,0,0.000000,1


I think since theres 44 possible teammates (44 total characters, 43 possible teammate characters), that I need 44 additional column headers, so that this can all be added  

Doing another pivot table then to make the teammate heroes the column headers and the teammate hero winrates as the values 

In [23]:
teammate_hero_df_w_synergy_PIVOT = teammate_hero_df_w_synergy.pivot_table(index=['player_id', 'hero_name'], columns='teammate_hero', values='teammate_hero_winrate').reset_index()
teammate_hero_df_w_synergy_PIVOT


teammate_hero,player_id,hero_name,Adam Warlock,Angela,Black Panther,Black Widow,Blade,Bruce Banner,Captain America,Cloak & Dagger,...,Squirrel Girl,Star-Lord,Storm,The Punisher,The Thing,Thor,Ultron,Venom,Winter Soldier,Wolverine
0,' F ',Doctor Strange,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,' noah,Gambit,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,''Nix'',Emma Frost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,''Nix'',Groot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,''Nix'',Luna Snow,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24148,ｚаｚа,Groot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24149,ｚаｚа,Invisible Woman,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
24150,ｚаｚа,Magneto,NaN,1.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
24151,ｚаｚа,Rocket Raccoon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


I have to open this in a csv to figure out the heroes that are missing (there are some heroes that people simply dont touch in top level play, at least for the 536 players i checked, but i still have to include their values and fill them with nan or all 0s)

In [24]:
teammate_hero_df_w_synergy_PIVOT.to_csv("teammate_hero_synergy_winrates.csv", index=False)

In [25]:
all_hero_names_in_data = ["Adam Warlock","Angela","Black Panther","Black Widow","Blade","Bruce Banner","Captain America","Cloak & Dagger","Daredevil","Doctor Strange","Emma Frost","Gambit","Groot","Hawkeye","Hela","Human Torch","Invisible Woman","Iron Fist","Iron Man","Jeff The Land Shark","Loki","Luna Snow","Magik","Magneto","Mantis","Mister Fantastic","Moon Knight","Namor","Peni Parker","Phoenix","Psylocke","Rocket Raccoon","Scarlet Witch","Spider-Man","Squirrel Girl","Star-Lord","Storm","The Punisher","The Thing","Thor","Ultron","Venom","Winter Soldier","Wolverine"]


there are no heroes that are missing, all 44 are there. I think that the "teammate_hero" column thats showing up is just an index column but its labeled for some reason 

I actually checked the csv file the "teammate hero" above the index is a visual error from vscode bc its not actually there in the csv 

Will fill in nans with heroes overall winrate again

In [26]:
#first merging all this data together with the hero_data_w_map_mode df 
final_hero_features = hero_data_w_map_mode.merge(teammate_hero_df_w_synergy_PIVOT, on=['player_id', 'hero_name'], how='left')
final_hero_features

,player_id,hero_name,role,timePlayed_value,matchesWinPct_value,matchesPlayed_value,kdRatio_value,kdaRatio_value,totalHeroDamagePerMinute_value,totalHeroHealPerMinute_value,...,Squirrel Girl,Star-Lord,Storm,The Punisher,The Thing,Thor,Ultron,Venom,Winter Soldier,Wolverine
0,VP Sypeh,Gambit,strategist,10.385762,0.691358,48.6,4.075556,9.626667,881.0,2983.0,...,NaN,0.500000,1.0,0.5,NaN,0.0,NaN,1.0,0.500000,0.750000
1,VP Sypeh,Invisible Woman,strategist,4.102634,0.639344,18.3,4.962500,11.475000,1294.0,3169.0,...,NaN,1.000000,NaN,NaN,NaN,1.0,NaN,0.0,0.666667,0.666667
2,VP Sypeh,Emma Frost,vanguard,0.354180,0.714286,1.4,3.363636,4.545455,2404.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VP Sypeh,Loki,strategist,1.963736,0.932203,11.8,5.400000,13.400000,957.0,2267.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,VP Sypeh,Adam Warlock,strategist,0.615783,0.533333,3.0,2.750000,4.850000,1272.0,1596.0,...,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3952,DecaDonald,Loki,strategist,10.502265,0.572016,48.6,2.886207,7.500000,701.0,2287.0,...,NaN,0.714286,0.0,NaN,NaN,0.0,NaN,0.5,0.333333,0.000000
3953,DecaDonald,Invisible Woman,strategist,2.657516,0.336283,11.3,2.098901,5.571429,677.0,2592.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
3954,DecaDonald,Rocket Raccoon,strategist,3.638904,0.590674,19.3,1.400000,6.410526,207.0,1988.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,1.0,0.000000,NaN
3955,DecaDonald,Venom,vanguard,0.448317,0.400000,2.5,2.117647,2.941176,1095.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000


In [27]:
#filling in nans with overall hero winrate again
all_heroes = ['Adam Warlock',
'Angela',
'Black Panther',
'Black Widow',
'Blade',
'Bruce Banner',
'Captain America',
'Cloak & Dagger',
'Daredevil',
'Doctor Strange',
'Emma Frost',
'Gambit',
'Groot',
'Hawkeye',
'Hela',
'Human Torch',
'Invisible Woman',
'Iron Fist',
'Iron Man',
'Jeff The Land Shark',
'Loki',
'Luna Snow',
'Magik',
'Magneto',
'Mantis',
'Mister Fantastic',
'Moon Knight',
'Namor',
'Peni Parker',
'Phoenix',
'Psylocke',
'Rocket Raccoon',
'Scarlet Witch',
'Spider-Man',
'Squirrel Girl',
'Star-Lord',
'Storm',
'The Punisher',
'The Thing',
'Thor',
'Ultron',
'Venom',
'Winter Soldier',
'Wolverine']


for hero in all_heroes:
    final_hero_features[hero] = final_hero_features[hero].fillna(0.5) #0.5 is basically not much of a synergy impact, its a 50/50 chance to win 
final_hero_features

,player_id,hero_name,role,timePlayed_value,matchesWinPct_value,matchesPlayed_value,kdRatio_value,kdaRatio_value,totalHeroDamagePerMinute_value,totalHeroHealPerMinute_value,...,Squirrel Girl,Star-Lord,Storm,The Punisher,The Thing,Thor,Ultron,Venom,Winter Soldier,Wolverine
0,VP Sypeh,Gambit,strategist,10.385762,0.691358,48.6,4.075556,9.626667,881.0,2983.0,...,0.5,0.500000,1.0,0.5,0.5,0.0,0.5,1.0,0.500000,0.750000
1,VP Sypeh,Invisible Woman,strategist,4.102634,0.639344,18.3,4.962500,11.475000,1294.0,3169.0,...,0.5,1.000000,0.5,0.5,0.5,1.0,0.5,0.0,0.666667,0.666667
2,VP Sypeh,Emma Frost,vanguard,0.354180,0.714286,1.4,3.363636,4.545455,2404.0,0.0,...,0.5,0.500000,0.5,0.5,0.5,0.5,0.5,0.5,0.500000,0.500000
3,VP Sypeh,Loki,strategist,1.963736,0.932203,11.8,5.400000,13.400000,957.0,2267.0,...,0.5,0.500000,0.5,0.5,0.5,0.5,0.5,0.5,0.500000,0.500000
4,VP Sypeh,Adam Warlock,strategist,0.615783,0.533333,3.0,2.750000,4.850000,1272.0,1596.0,...,0.5,1.000000,0.5,0.5,0.5,0.5,0.5,0.5,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3952,DecaDonald,Loki,strategist,10.502265,0.572016,48.6,2.886207,7.500000,701.0,2287.0,...,0.5,0.714286,0.0,0.5,0.5,0.0,0.5,0.5,0.333333,0.000000
3953,DecaDonald,Invisible Woman,strategist,2.657516,0.336283,11.3,2.098901,5.571429,677.0,2592.0,...,0.5,0.500000,0.5,0.5,0.5,0.5,0.5,0.5,0.000000,0.500000
3954,DecaDonald,Rocket Raccoon,strategist,3.638904,0.590674,19.3,1.400000,6.410526,207.0,1988.0,...,0.5,0.500000,0.5,0.0,0.0,0.5,0.5,1.0,0.000000,0.500000
3955,DecaDonald,Venom,vanguard,0.448317,0.400000,2.5,2.117647,2.941176,1095.0,0.0,...,0.5,0.500000,0.5,0.5,0.5,0.5,0.5,0.5,0.500000,1.000000


In [28]:
final_hero_features.to_csv("final_hero_dataset.csv", index=False)